In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
from tensorflow import data as tf_data
from tensorboard.plugins.hparams import api as hp
import keras_tuner as kt

from sklearn.model_selection import train_test_split

from preprocess_common import *

from record_save_load import *

In [2]:
class_names=['Human', 'AI']
AUTO = tf_data.AUTOTUNE # Parallelize data loading
#Hyperparameters
BATCH_SIZE = 64
BUFFER_SIZE = 1024
IMG_SIZE = (380,380)

SEED = 44

In [3]:
eff_train_ds = load_tfrecord("records/eff_ens_train.tfrecord", BATCH_SIZE, "efficientnet").repeat()
eff_val_ds = load_tfrecord("records/eff_ens_val.tfrecord", BATCH_SIZE, "efficientnet")
res_train_ds = load_tfrecord("records/res_ens_train.tfrecord", BATCH_SIZE, "resnet").repeat()
res_val_ds = load_tfrecord("records/res_ens_val.tfrecord", BATCH_SIZE, "resnet")

In [4]:
res_model=keras.saving.load_model("./models/resnet_tuned_best_v32131289_blur_nomix_focal.keras")
eff_model=keras.saving.load_model("./models/efficientnet_tuned_blur_no_cutmix.keras")

In [ ]:
#fit pred into models and build datasets
res_pred_train_x=res_model.predict(res_train_ds)
res_pred_val_x=res_model.predict(res_val_ds)
eff_pred_train_x=eff_model.predict(eff_train_ds)
eff_pred_val_x=eff_model.predict(eff_val_ds)
ens_train_ds=tf_data.Dataset.from_tensor_slices((res_pred_train_x,eff_pred_train_x,res_train_ds))
ens_val_ds=tf_data.Dataset.from_tensor_slices((res_pred_val_x,eff_pred_val_x,res_val_ds))
ens_train_ds=ens_train_ds.map(lambda x,y,z: (tf.convert_to_tensor((float(x[0]),float(y[0]))),z))
ens_val_ds=ens_val_ds.map(lambda x,y,z: (tf.convert_to_tensor((float(x[0]),float(y[0]))),z))

   9311/Unknown 19381s 2s/step

KeyboardInterrupt: 

In [ ]:
#Build Model
input=keras.layers.Input(shape=(2,))
x=keras.layers.Dense(8)(input)
x = keras.layers.Dropout(0.1, seed=SEED)(x)
outputs = keras.layers.Dense(2, activation='softmax')(x)
ens_model=keras.Model(input,outputs)
lr_schedule = keras.optimizers.schedules.CosineDecayRestarts(
        initial_learning_rate=9e-5,
        first_decay_steps=20, 
        t_mul=2.0,  # Restarts period multiplier
        m_mul=0.9  # Decay factor after each restart
    )
ens_model.compile(optimizer=keras.optimizers.AdamW(lr_schedule),
                    loss=keras.losses.CategoricalFocalCrossentropy(alpha=1.0),
                    metrics=[keras.metrics.AUC(multi_label=True),
                            keras.metrics.F1Score("macro")])
early_stopping = keras.callbacks.EarlyStopping(monitor="val_f1_score", patience=2, min_delta=0.01, mode="max")
save_model = keras.callbacks.ModelCheckpoint(f'ens_res_eff.keras', save_best_only=True)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir="logs/ensemble")
ens_model.fit(
    ens_train_ds,
    validation_data=ens_val_ds,
    epochs=10,
    steps_per_epoch=10,
    callbacks=[early_stopping,
               save_model,
               tensorboard_callback]
)

Epoch 1/10


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("data:0", shape=(2,), dtype=float32). Expected shape (None, 2), but input has incompatible shape (2,)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(2,), dtype=float32)
  • training=True
  • mask=None